<a href="https://colab.research.google.com/github/Fat-AK/Deep-Learning-/blob/main/Copy_of_HMW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. MDPs

## 1.1 chess

 chess board :  3x3 grid with only the kings on the board. 
 The goal of the game : checkmate the opponent's king or force the opponent to resign. The player who achieves this goal is considered the winner.

State state (s): All possible board positions, players, and any additional information. In this case, there are nine possible board positions, two players (white and black), and a turn counter.

Action Space (a): All legal moves that can be made from the current board position. In this case, each player can move their king to any adjacent cell on the board, or they can choose to pass their turn.

Reward Function (r): Win +1, Loss -1, and 0 for a draw or passing the turn.

Transition Probability: The transition probability would represent the likelihood of moving from one board position to another after a move is made. In this case, the transition probability would be deterministic since there are only a limited number of possible moves from each board position.

Discount Factor: The discount factor is a value between 0 and 1 that represents the importance of future rewards compared to immediate rewards. In chess, a discount factor of 1 can be used because the outcome of the game is determined at the end.

## 1.2 LunarLander

The link does not work 

## 1.3 Model Based RL: Accessing Environment Dynamics

### Discuss the Policy Evaluation and Policy Iteration algorithms from the lecture.They explicitly make use of the environment dynamics (p(s′, r|s, a)).

Policy evaluation is the process of calculating the value function for a given policy (π) in an MDP. The value function determines the expected return (Gt) for an agent starting in a particular state (s) and following a specific policy. The policy evaluation algorithm estimates the value function iteratively, starting from an arbitrary initial value function and updating it until it converges to the true value function.

1- Initialize the value function arbitrarily
Repeat:
    For each state s:
    Update the value function V(s) = Σ p(s', r|s, a) [r + γV(s')]
    Where p(s', r|s, a) is the probability of transitioning to state s' and receiving reward r when taking action a from state s, and γ is the discount factor.
    
Policy Iteration:

Policy iteration is an RL algorithm that alternates between policy evaluation and policy improvement to find the optimal policy for an MDP. The algorithm starts with an arbitrary policy and iteratively updates it until the optimal policy is found.

The key step in improving the policy was understanding we could compute the optimal action given the current policy and state value by looking at the expected return of taking any action a in a given state and afterwards following the current policy:
∑s′,r[p(s′,r|s,a)(r+Vπ(s′))]

if we compare the expected Return of all possible different actions in a given state, we can just pick the best one to optimize the policy:
π(s)=argmaxa[∑s′,r[p(s′,r|s,a)(r+Vπ(s′))]]


### Explain what the environment dynamics (i.e. reward function and state transition function) are and give at least two examples.

The environment dynamics in a reinforcement learning problem refer to the reward function and state transition function, which determine the consequences of an agent's actions in the environment.

Reward Function: The reward function maps a state and action pair to a numerical reward value that represents how good or bad that action was for the agent. The goal of the agent is to learn a policy that maximizes the cumulative reward it receives over time. A well-designed reward function is critical for guiding the agent towards desirable behavior.

Example 1: Consider a self-driving car navigating through traffic. A reward function might assign positive rewards for reaching the destination safely and quickly, negative rewards for causing accidents or violating traffic laws, and intermediate rewards for following a smooth and comfortable driving style. The reward function should be designed to prioritize safety, as that is the most important objective.

Example 2: In a game of chess, the reward function might assign a positive reward for winning the game, a negative reward for losing, and a zero reward for a draw. The reward function could also assign intermediate rewards for taking control of the center of the board, putting the opponent's king in check, or capturing the opponent's pieces.

State Transition Function: The state transition function maps a current state and action pair to a next state. It determines how the environment evolves over time in response to the agent's actions. In many reinforcement learning problems, the state transition function is stochastic, meaning that the next state is not entirely predictable and can depend on random factors.



### Discuss: Are the environment dynamics generally known and can practically be used to solve a problem with RL?


#  2.Implementing a GridWorld

### 2.1 Look up some examples of GridWorld! List at least three links or references to different GridWorlds you could find online

1.   https://cs.stanford.edu/people/karpathy/reinforcejs/gridworld_dp.html
2.   https://taigame.org/game/gridworld
3. https://strategywiki.org/wiki/Photon:_The_Ultimate_Game_on_Planet_Earth/Walkthrough



### 2.2 Implementing the MDP

 Implement a GridWorld Class

In [38]:
import numpy as np

class GridWorld:
    def __init__(self, size, rewards, discount_factor):
        self.size = size
        self.rewards = rewards
        self.discount_factor = discount_factor
        
        self.num_states = size[0] * size[1]
        self.num_actions = 4 # up, down, left, right
        
        # create transition probability matrix
        self.transition_probs = np.zeros((self.num_states, self.num_actions, self.num_states))
        for s in range(self.num_states):
            for a in range(self.num_actions):
                next_s = self._get_next_state(s, a)
                self.transition_probs[s, a, next_s] = 1
        
    def _get_next_state(self, state, action):
        x, y = state // self.size[1], state % self.size[1]
        if action == 0: # up
            x = max(x-1, 0)
        elif action == 1: # down
            x = min(x+1, self.size[0]-1)
        elif action == 2: # left
            y = max(y-1, 0)
        elif action == 3: # right
            y = min(y+1, self.size[1]-1)
        return x * self.size[1] + y
        
    def get_transition_probs(self):
        return self.transition_probs
    
    def get_rewards(self):
        return self.rewards
    
    def get_discount_factor(self):
        return self.discount_factor
    
    def get_num_states(self):
        return self.num_states
    
    def get_num_actions(self):
        return self.num_actions


In this example, a simple 3x3 Gridworld MDP, the transition probabilities are ***deterministic***. This means that taking an action in a particular state always results in the same next state. For example, if the agent is in state 1 and takes the "right" action, it will transition to state 2 with probability 1. Therefore, this MDP is ***deterministic***.

In [39]:
import numpy as np

class GridWorld:
    def __init__(self, size, rewards, discount_factor, walls=[], traps=[]):
        self.size = size
        self.rewards = rewards
        self.discount_factor = discount_factor
        self.walls = walls
        self.traps = traps
        
        self.num_states = size[0] * size[1]
        self.num_actions = 4 # up, down, left, right
        
        # create transition probability matrix
        self.transition_probs = np.zeros((self.num_states, self.num_actions, self.num_states))
        for s in range(self.num_states):
            for a in range(self.num_actions):
                next_s = self._get_next_state(s, a)
                self.transition_probs[s, a, next_s] = 1
        
    def _get_next_state(self, state, action):
        x, y = state // self.size[1], state % self.size[1]
        if action == 0: # up
            if x > 0 and (x-1, y) not in self.walls:
                x = x - 1
        elif action == 1: # down
            if x < self.size[0]-1 and (x+1, y) not in self.walls:
                x = x + 1
        elif action == 2: # left
            if y > 0 and (x, y-1) not in self.walls:
                y = y - 1
        elif action == 3: # right
            if y < self.size[1]-1 and (x, y+1) not in self.walls:
                y = y + 1
                
        next_state = x * self.size[1] + y
        
        # check for traps
        if next_state in self.traps:
            return 0 # reset to initial state
        else:
            return next_state
        
    def get_transition_probs(self):
        return self.transition_probs
    
    def get_rewards(self):
        return self.rewards
    
    def get_discount_factor(self):
        return self.discount_factor
    
    def get_num_states(self):
        return self.num_states
    
    def get_num_actions(self):
        return self.num_actions


# 3  Implementing a policy 

### 3.1 Implement the basic agent


In [40]:
import numpy as np

class StochasticAgent:    # 
    def __init__(self, env, stochasticity=0.2):
        self.env = env
        self.stochasticity = stochasticity
    
    def act(self, state):
        """
        Given the current state, return an action according to a stochastic policy
        with non-zero probability for every action at every state.
        """
        action_probs = np.ones(self.env.get_num_actions()) * self.stochasticity / (self.env.get_num_actions() - 1)
        action_probs[self.env.get_num_actions()//2] = 1 - self.stochasticity # set the probability of the action that corresponds to staying in place to 1 - stochasticity
        
        action = np.random.choice(self.env.get_num_actions(), p=action_probs)
        return action


### 3.2  Evaluate the policy 

In [41]:
pip install gridworld


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/path/to/gridworld']


In [44]:
import sys
sys.path.append("/path/to/gridworld")

In [ ]:
import numpy as np
from gridworld import GridWorld
from agent import StochasticAgent

# create the GridWorld and agent
gw_size = (5, 5)
gw_rewards = np.zeros(gw_size)
gw_rewards[gw_size[0]-1, gw_size[1]-1] = 1 # reward at goal state
gw_discount = 0.99
gw = GridWorld(gw_size, gw_rewards, gw_discount)
agent = StochasticAgent(gw.get_num_actions(), gw.get_num_states())

# collect episodes
num_episodes = 1000
state_values = np.zeros(gw.get_num_states()) # state value function V(s)
state_counts = np.zeros(gw.get_num_states()) # state visitation counts
for i in range(num_episodes):
    # initialize episode
    state = np.random.randint(gw.get_num_states())
    episode = []
    done = False
    while not done:
        # take action according to the agent's policy
        action = agent.act(state)
        next_state, reward = gw.step(state, action)
        episode.append((state, action, reward))
        state = next_state
        done = (reward == 1) # reached goal state
    # update state value function
    returns = 0
    for j in range(len(episode)-1, -1, -1): # iterate over episode backwards
        state, action, reward = episode[j]
        returns = gw.get_discount_factor() * returns + reward
        if state_counts[state] == 0: # first visit to the state
            state_values[state] = returns
        else: # incremental update
            state_values[state] = (state_values[state] * state_counts[state] + returns) / (state_counts[state] + 1)
        state_counts[state] += 1

# print state values
print("State values:")
print(np.reshape(state_values, gw_size))
